<a href="https://colab.research.google.com/github/Starsa/FunDataProjects/blob/main/27_10_NLP_YodaChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YODA


> **"DO. OR DO NOT. THERE IS NO TRY.!"**

Ever want to have a conversation with yoda? I created a chatot that responds in a similar format as Yoda.

I found the corpus of dialogue [here.](https://www.kaggle.com/stefanocoretta/yoda-speech-corpus?select=yoda-corpus.csv)

### Import libraries for use

In [ ]:
#mount google drive so for access to csv with dataset

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#uninstall then re-install spacy for use
#some issues with installing certain libraries
#!pip uninstall spacy
#!pip install spacy

In [ ]:
#install chatter bot
!pip install chatterbot
!pip install chatterbot-corpus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 1.3 MB 10.9 MB/s 
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.39
    Uninstalling SQLAlchemy-1.4.39:
      Successfully uninstalled SQLAlchemy-1.4.39
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 117 kB 5.4 MB/s 


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
import spacy
import re
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer
from chatterbot.conversation import Statement
import random

import warnings
warnings.filterwarnings(action="ignore")

!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Create dataframe for cleanup and EDA

In [ ]:
#create dataset to work with
yoda= pd.read_csv('/content/drive/My Drive/Thinkful/NLP/yoda-corpus 2.csv')

#make a copy that is clean
yoda_Restore = yoda.copy(deep=True)
yoda.shape

yoda.head()

,movie,scene,line,character,text,slug,component
0,1,129,1158,narrator,QUI-GON stands in a tall stately room. Twelve ...,INT. TEMPLE OF THE JEDI - COUNCIL CHAMBERS - DAY,action
1,1,129,1159,QUI-GON,...my only conclusion can be that it was a Sit...,INT. TEMPLE OF THE JEDI - COUNCIL CHAMBERS - DAY,character
2,1,129,1160,MACE WINDU,A Sith Lord?!?,INT. TEMPLE OF THE JEDI - COUNCIL CHAMBERS - DAY,character
3,1,129,1161,KI-ADI,Impossible! The Sith have been extinct for a m...,INT. TEMPLE OF THE JEDI - COUNCIL CHAMBERS - DAY,character
4,1,129,1162,YODA,"The very Republic is threatened, if involved t...",INT. TEMPLE OF THE JEDI - COUNCIL CHAMBERS - DAY,character


#### Review characters and engineer yoda to 1 and everyone else to 0

In [ ]:
yoda['character'] = yoda['character'].str.strip().str.lower()
yoda['character'] = yoda['character'].replace(("yoda\t\t (gathering all his strength)","yoda\t (shakes his head)", "yoda\t (tickled, chuckles)"),
                                              "yoda").replace("ki-adi", "kl-adi-mundi")
yoda.character.value_counts()

yoda                    103
narrator                 91
obi-wan                  41
anakin                   26
mace windu               21
qui-gon                  15
mace                     15
kl-adi-mundi             12
bail organa              11
luke                      9
darth sldious             6
creature                  3
medical droid             3
clone sergeant            2
clone commander gree      2
pilot                     2
mas amedda                2
mace wlndu                1
clone commander cody      1
clone trooper             1
qui -gon                  1
c-3po                     1
at-st clone sergeant      1
Name: character, dtype: int64

In [ ]:
yoda['character'] = np.where(yoda['character']== "yoda",1,0)
yoda.character.value_counts()

0    268
1    103
Name: character, dtype: int64

In [ ]:
yoda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movie      371 non-null    int64 
 1   scene      371 non-null    int64 
 2   line       371 non-null    int64 
 3   character  371 non-null    int64 
 4   text       371 non-null    object
 5   slug       371 non-null    object
 6   component  371 non-null    object
dtypes: int64(4), object(3)
memory usage: 20.4+ KB


### Create a new dataframe for the chattbot with only Yoda dialogue.

In [ ]:
yoda_speech = yoda.loc[yoda["character"]==1]
yoda_speech = yoda_speech[["text", "character"]]
yoda_speech.shape

(103, 2)

In [ ]:
yoda_speech.head(20)

,text,character
4,"The very Republic is threatened, if involved t...",1
6,"Hard to see, the dark side is. Discover who th...",1
9,"With this Naboo queen you must stay, Qui-Gon. ...",1
11,May the Force be with you.,1
13,(Cont'd) Master Qui-Gon more to say have you?,1
15,"A vergence, you say?",1
20,But you do! Rrevealed your opinion is.,1
23,"Trained as a Jedi, you request for him?",1
26,Tested he will be.,1
30,"Good, good, young one. How feel you?",1


### Use NLP to generate text

In [ ]:
#Note: When parsing the data using SpaCy, you may run into some memory issues even in Google Colaboratory.
#If you're having memory issues, try parsing your text as follows:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp.add_pipe('sentencizer')
nlp.max_length = 20000000
yoda_doc = nlp(" ".join(yoda_speech.text))

In [ ]:
yoda_sents = [sent.text for sent in yoda_doc.sents if len(sent.text) > 1]

#### Create greetings for bot

In [ ]:
GREETING_INPUTS = ["hello", "hi", "greetings", "what's up","hey"]
GREETING_RESPONSES = ["hello, I will say", "Do. Or do not. There is no try.!", "here yoda is", "hello"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

#### Train the bot

In [ ]:
# create a chatbot
chatbot = ChatBot('Yoda')
# this is to remove the accumulated knowledge base
chatbot.storage.drop()

# create a new trainer for the chatbot
trainer = ListTrainer(chatbot)

# train the chatbot based on Yoda dialog
trainer.train(yoda_sents)

OSError: ignored

In [ ]:
import random

# Chat with YODA

In [ ]:
print("Yoda: Respond to you, I will. If exit you want, type you will 'bye'.")

# below is the chatting
while True:

    user_input = input("User: ")
    user_input=user_input.lower()

    if(user_input!='bye'):
        if(user_input == 'thankful I am' or user_input == 'YODA-best, thank you I say'):
            break
            print("Yoda: Welcome you are.")
        else:
            if(greeting(user_input) != None):
                print("Yoda: " + greeting(user_input))
            else:
                print("Yoda: ", end = "")
                print(chatbot.get_response(user_input))
    else:
        print("Yoda: Goodbye, I must say. Miss you I will.")
        break